## Optimizated Version with MLX 

In [6]:
import PyPDF2
import csv
import random
from tqdm import tqdm
import time
from transformers import AutoTokenizer
<<<<<<< Tabnine <<<<<<<
def run_command_with_live_output(command: list[str]) -> None:#+
    """#+
    This function executes a command and prints its live output.#+
#+
    Parameters:#+
    command (list[str]): A list of strings representing the command to be executed.#+
#+
    Returns:#+
    None: The function does not return any value. It prints the live output of the command.#+
    """#+
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)#+
    while True:#+
        output = process.stdout.readline()#+
        if output == '' and process.poll() is not None:#+
            break#+
        if output:#+
            print(output.strip())#+
    err_output = process.stderr.read()#+
    if err_output:#+
        print(err_output)#+
>>>>>>> Tabnine >>>>>>>
from mlx_lm import load, generate
import subprocess

def run_command_with_live_output(command: list[str]) -> None:
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    err_output = process.stderr.read()
    if err_output:
        print(err_output)

def extract_text_from_pdf(pdf_path):
    print("Extracting text from PDF...")
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in tqdm(reader.pages, desc="Processing pages"):
            text += page.extract_text()
    print("Text extraction completed")
    return text

def create_chunks(text, min_chunk_size=200, max_chunk_size=500):
    print("Splitting text into chunks...")
    chunks = []
    current_chunk = ""
    sentences = text.split('.')
    for sentence in tqdm(sentences, desc="Creating chunks"):
        if len(current_chunk) + len(sentence) > max_chunk_size and len(current_chunk) >= min_chunk_size:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            current_chunk += sentence + '.'
    if current_chunk:
        chunks.append(current_chunk.strip())
    print(f"Number of chunks created: {len(chunks)}")
    return chunks

def load_model_and_tokenizer(model_path):
    print("Loading model and tokenizer...")
    model, tokenizer = load(model_path)
    return model, tokenizer

def generate_question_response_pair(chunk, model, tokenizer, max_tokens=512):
    instruction = """
    Based on the compliance requirements by CHERRY shipping line, 
    create a concise question and provide a brief, informative answer. 
    All the questions should be based on a country if it is required per provided documentation.
    """
    prompt = f'''<s>[INST] {instruction}\n\nText: {chunk}\n\nQuestion: [/INST]\n'''
    
    generated_text = generate(model, tokenizer, prompt=prompt, max_tokens=max_tokens, verbose=False)
    generated_text = generated_text.replace(prompt, "").strip()
    parts = generated_text.split("\n\nAnswer:", 1)
    
    if len(parts) == 2:
        question, response = parts
    else:
        question = "Unable to generate a question."
        response = "Unable to generate a response."
    
    return question.strip(), response.strip()

def create_fine_tuning_dataset(pdf_path, output_file, num_samples=10000):
    start_time = time.time()
    print(f"Starting fine-tuning dataset creation (Target samples: {num_samples})")

    model_path = "mlx-community/Mistral-7B-Instruct-v0.2-4bit"
    # model_path = "mlx-community/gemma-1.1-7b-it-4bit"
    # model_path = "mlx-community/quantized-gemma-2b-it"
    # model_path = "mlx-community/quantized-gemma-7b-it"
    # model_path = "mlx-community/gemma-2-27b-it-4bit"
    model, tokenizer = load_model_and_tokenizer(model_path)
    print("Model and tokenizer loaded")

    text = extract_text_from_pdf(pdf_path)
    chunks = create_chunks(text)
    
    print("Generating question-response pairs and writing to CSV...")
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['question', 'response'])
        
        for _ in tqdm(range(num_samples), desc="Generating samples"):
            chunk = random.choice(chunks)
            question, response = generate_question_response_pair(chunk, model, tokenizer)
            writer.writerow([question, response])
            if (_ + 1) % 100 == 0:
                print(f"Generated {_ + 1} samples...")
    
    end_time = time.time()
    print(f"Number of dataset samples generated: {num_samples}")
    print(f"Total time taken: {end_time - start_time:.2f} seconds")

# Usage example
pdf_path = "CHERRY_compliance.pdf"
output_file = "CHERRY_compliance.csv"
create_fine_tuning_dataset(pdf_path, output_file, num_samples=10000)

Starting fine-tuning dataset creation (Target samples: 10000)
Loading model and tokenizer...


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Model and tokenizer loaded
Extracting text from PDF...


Processing pages: 100%|██████████| 206/206 [00:01<00:00, 105.23it/s]


Text extraction completed
Splitting text into chunks...


...

Generated 9800 samples...


Generating samples:  99%|█████████▉| 9900/10000 [7:45:52<04:32,  2.72s/it]

Generated 9900 samples...


Generating samples: 100%|██████████| 10000/10000 [7:50:49<00:00,  2.82s/it]

Generated 10000 samples...
Number of dataset samples generated: 10000
Total time taken: 28260.52 seconds


## Generation with Gemma model

In [ ]:
import PyPDF2
import csv
import random
from tqdm import tqdm
import time
from transformers import AutoTokenizer
from mlx_lm import load, generate
import subprocess

def run_command_with_live_output(command: list[str]) -> None:
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    err_output = process.stderr.read()
    if err_output:
        print(err_output)

def extract_text_from_pdf(pdf_path):
    print("Extracting text from PDF...")
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in tqdm(reader.pages, desc="Processing pages"):
            text += page.extract_text()
    print("Text extraction completed")
    return text

def create_chunks(text, min_chunk_size=200, max_chunk_size=1000):
    print("Splitting text into chunks...")
    chunks = []
    current_chunk = ""
    sentences = text.split('.')
    for sentence in tqdm(sentences, desc="Creating chunks"):
        if len(current_chunk) + len(sentence) > max_chunk_size and len(current_chunk) >= min_chunk_size:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            current_chunk += sentence + '.'
    if current_chunk:
        chunks.append(current_chunk.strip())
    print(f"Number of chunks created: {len(chunks)}")
    return chunks

def load_model_and_tokenizer(model_path):
    print("Loading model and tokenizer...")
    model, tokenizer = load(model_path)
    return model, tokenizer

def generate_question_response_pair(chunk, model, tokenizer, max_tokens=512):
    instruction = """
    Based on the compliance requirements by CHERRY shipping line, 
    create a concise question and provide a brief, informative answer. 
    All the questions should be based on a country if it is required per provided documentation.
    Do not include any special markers like '**' or '<end_of_turn>'
    Format your response as follows:
    Question: [Your generated question]
    Answer: [Your generated answer]
    """
    prompt = f'''<s>[INST] {instruction}\n\nText: {chunk} [/INST]\n'''
    
    generated_text = generate(model, tokenizer, prompt=prompt, max_tokens=max_tokens, verbose=False)
    generated_text = generated_text.replace(prompt, "").strip()
    generated_text = re.sub(r'\*\*|<end_of_turn>', '', generated_text)
    
    question_start = generated_text.find("Question:")
    answer_start = generated_text.find("Answer:")
    
    if question_start != -1 and answer_start != -1:
        question = generated_text[question_start+9:answer_start].strip()
        answer = generated_text[answer_start+7:].strip()
    else:
        question = "Unable to generate a question."
        answer = "Unable to generate a response."
    
    return question, answer

def create_fine_tuning_dataset(pdf_path, output_file, num_samples=10000):
    start_time = time.time()
    print(f"Starting fine-tuning dataset creation (Target samples: {num_samples})")

    model_path = "mlx-community/gemma-2-27b-it-4bit"
    model, tokenizer = load_model_and_tokenizer(model_path)
    print("Model and tokenizer loaded")

    text = extract_text_from_pdf(pdf_path)
    chunks = create_chunks(text)
    
    print("Generating question-response pairs and writing to CSV...")
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['question', 'response'])
        
        for _ in tqdm(range(num_samples), desc="Generating samples"):
            chunk = random.choice(chunks)
            question, response = generate_question_response_pair(chunk, model, tokenizer)
            if question != "Unable to generate a question." and response != "Unable to generate a response.":
                writer.writerow([question, response])
            else:
                print(f"Skipping invalid sample: Q: {question}, A: {response}")
            if (_ + 1) % 100 == 0:
                print(f"Generated {_ + 1} samples...")
    
    end_time = time.time()
    print(f"Number of dataset samples generated: {num_samples}")
    print(f"Total time taken: {end_time - start_time:.2f} seconds")

# Usage example
pdf_path = "./data/CHERRY_compliance.pdf"
output_file = "./data/CHERRY_compliance.csv"
create_fine_tuning_dataset(pdf_path, output_file, num_samples=10000)